In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys

print(os.getcwd())

/content


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers # >= 4.x.x
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680533 sha256=87521190b0f413abb4999b2c3ae41cd395f1aea4d04c398cd6c839d06d697958
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-jvulgg_5
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-jvulgg_5
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd
chatbot_data = pd.read_csv('/content/sentiment_dialogues.csv')

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
chatbot_data.sample(n=10)

In [ ]:
df = chatbot_data.drop(['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', '공포', '5468'], axis=1)

In [ ]:
df.tail(10)

,발화,감정
146557,빨갱이 종북 박사모 알바들이 아주 여어엄벼엉을 하고 있네.,혐오
146558,트럼프는 박그네한테 고마워할듯? ㅋㅋ,혐오
146559,정신병자새끼 밥되게 놔두지!!,혐오
146560,결국에는 아베가 짜놓은 판에 놀아나고 있는거네,혐오
146561,"비박, 너희들도 자격없다.",혐오
146562,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
146563,재미가 없으니 망하지,혐오
146564,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
146565,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오
146566,와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요,혐오


In [ ]:
add_data1 = pd.read_csv('/content/추가 데이터셋 1.csv')
add_data2 = pd.read_csv('/content/추가 데이터셋 2.csv')

add_data1.head(10)

In [ ]:
add_data1 = add_data1.rename(columns={'Text': '발화', 'Sentiment': '감정'})
add_data2 = add_data2.rename(columns={'Text': '발화', 'Sentiment': '감정'})

In [ ]:
df_concat = pd.concat([df, add_data1], axis=0)
df_concat = pd.concat([df_concat, add_data2], axis=0)
df_concat.sample(10)

,발화,감정
111,그녀가 나에게 사귀자고 고백했어. 나도 원하고 있었는데 지금 기분이 너무 좋아.일단...,행복
89955,아저씨. 이거. 30만원씩이나 주구 산거에요!!,분노
32729,학교 은사님이 돌아가셨어. 너무 슬퍼.내가 세상에서 제일 존경하던 분이야. 마음이 ...,슬픔
8183,남자친구가 있는데 몰래 소개팅을 했어.남자친구는 결혼 상대는 아닌 것 같아. 하지만...,당황
124680,무기징역에 전 재산 환수하는 꼴을 보고야 말겠다,분노
11562,국가검진 대장암 검사를 받았는데 대장암 의심이 된다고 했어.응. 그런데 정밀검사를 ...,불안
99058,팀장 빨리 들어오라 그래.,중립
20073,자율적으로 일하는 좋은 팀원들을 만나서 감사해.내 팀에는 스스로 열심히 하는 팀원들...,행복
21620,수학을 너무 못 봐서 선생님께 혼이 나서 속상해.점수를 올리고 싶은데 방법이 뭘까?,슬픔
7359,EX) 일봉이 5일이상 횡보한 후 전날 대비 2 상승한 종목들을 선정 및 매수 그 ...,중립


In [ ]:
chatbout_data = df_concat
len(chatbout_data)

156837

In [ ]:
chatbot_data.loc[(chatbot_data['감정'] == "불안"), '감정'] = 0  #불안 => 0
chatbot_data.loc[(chatbot_data['감정'] == "당황"), '감정'] = 1  #당황 => 1
chatbot_data.loc[(chatbot_data['감정'] == "분노"), '감정'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['감정'] == "슬픔"), '감정'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['감정'] == "중립"), '감정'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['감정'] == "행복"), '감정'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['감정'] == "혐오"), '감정'] = 6  #혐오 => 6

data_list = []
for q, label in zip(chatbot_data['발화'], chatbot_data['감정'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.', '2']
['최근에 업무가 너무 많이 늘어난 것 같아 힘들어.안 그래도 할 일 많은데 왜 자꾸 나한테 추가 업무를 주는지 모르겠다니깐.', '2']
['친구들은 다 취업에 성공했는데 나만 못한 것 같아.나도 졸업하고 바로 일하고 싶었는데.', '0']
['친구에게 내가 간암에 걸려서 술을 마실 수 없다고 하자 거짓말하지 말라고 했어.간암에 걸렸다는 말을 왜 안 믿을까?', '3']
['의사가 분명히 수술이 잘 되었다고 했거든. 그런데 삼 주가 되도록 몸을 움직일 수 없어.그러니 불안해서 견딜 수가 있어야지. 아예 움직일 수 없을까 봐.', '3']
['이번에 주식을 샀는데 주가가 너무 떨어져 손해가 이만저만이 아냐.지금은 없어. 난 왜 이렇게 되는일이 없나 몰라.', '3']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.13, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

127513
19054


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  assert self._num_workers == 0


In [ ]:
# for data in train_dataloader:
#   print(data)
#   break
#kobert 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))[1]
        #bert model returns 'last_hidden_state' and 'pooler_output'

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
# train_dataloader

In [ ]:
#kobert 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        with torch.no_grad():
            out = model(token_ids, valid_length, segment_ids)
        test_loss=loss_fn(out,label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {} test loss {}".format(e+1, test_acc / (batch_id+1),test_loss.data.cpu().numpy()))

    PATH = '/content/drive/MyDrive/Colab Notebooks/KHUDA 3기/감성분석/epoch{}_model.pt'.format(e+1)
    torch.save(model.state_dict(), PATH)

<ipython-input-60-412a5ece418c>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1993 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  assert self._num_workers == 0


epoch 1 batch id 1 loss 0.9896848797798157 train acc 0.609375
epoch 1 batch id 201 loss 0.8160349130630493 train acc 0.6692319651741293
epoch 1 batch id 401 loss 0.8216909766197205 train acc 0.6738622194513716
epoch 1 batch id 601 loss 0.9253036975860596 train acc 0.6749428036605657
epoch 1 batch id 801 loss 0.7981774806976318 train acc 0.6786243757802747
epoch 1 batch id 1001 loss 1.0179569721221924 train acc 0.6814279470529471
epoch 1 batch id 1201 loss 0.7285289168357849 train acc 0.6826602830974188
epoch 1 batch id 1401 loss 1.0165231227874756 train acc 0.6829385260528195
epoch 1 batch id 1601 loss 0.8665224313735962 train acc 0.68183947532792
epoch 1 batch id 1801 loss 0.8185220956802368 train acc 0.6838388395335925
epoch 1 train acc 0.6843220020070245


<ipython-input-60-412a5ece418c>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

  0%|          | 0/298 [00:00<?, ?it/s]

epoch 9 test acc 0.6938717354829297 test loss 1.2285844087600708


  0%|          | 0/1993 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.11984764784574509 train acc 0.953125
epoch 10 batch id 201 loss 0.22882504761219025 train acc 0.9556902985074627
epoch 10 batch id 401 loss 0.28170737624168396 train acc 0.9554629052369077
epoch 10 batch id 601 loss 0.20411323010921478 train acc 0.9553868552412645
epoch 10 batch id 801 loss 0.053227417171001434 train acc 0.9566167290886392
epoch 10 batch id 1001 loss 0.0764177143573761 train acc 0.957199050949051
epoch 10 batch id 1201 loss 0.08976487070322037 train acc 0.9573012073272273
epoch 10 batch id 1401 loss 0.19714908301830292 train acc 0.9578760706638115


KeyboardInterrupt: ignored

In [ ]:
chatbout_data.to_csv("total_data.csv", mode='w')